https://canvas.uw.edu/courses/1022940/assignments/3103586

* develop flexible code that lets us reclassify an attribute within a vector feature class according to flexible criteria.  

* package the code up into a geoprocessing tool that can be used within ArcGIS Desktop’s GUI, within ModelBuilder, or in ArcPy.  

* experiment with new ways of putting the tool you develop out onto the internet as a service that could be used by other people on other computers from afar without downloading or installing the code locally.  


it’s important for you to first learn not only about how tools are created, 
but about how they are published as services online in ArcGIS 10.3.x.

###### Goal: 
able to publish a tool as a geoprocessing service imposes structure on the way that the tool can accept inputs through its parameters.  
which will have implications for how you write parts of your code. At the very least, consider doing much of your initial reading for the various parts of the assignment first before coding the reclassification tool.


#####  task to be solved

You will need to design and write code to accomplish reclassification


- user gives you a valid shapefile, perhaps called mytable  
(though, in reality, you will be getting mytable from the user via the geoprocessing tool input parameters).

- user specifies that s/he wants to reclassify the [‘double’ type] field infield…
 …and store the results in a field that does not exist yet, outfield, whose name the user specifies.
    
- user provides you with a table called reclasstable that tells you how to do the reclassification.   
  Specifically, the table has columns named lowerbound, upperbound, and value.  

- For every row in mytable, your code must look at the value of infield and decide what to write in the outfield.

- therefore it must go through the reclasstable to find the first row in reclasstable where the value from infield is greater than or equal to the lowerbound and less-than-or-equal-to the upperbound.

- If this row exists, then outfield should be set to value.

- If no row in the reclasstable matches the current value in infield, then outfield should be set to a special number called notfoundvalue, which the user has also specified.

##### Sample input conditions and outputs

- Given this shapefileView in a new window of King County block groups, 
-  Reclassify the field "PopDens12" (corresponding to the population density, per square mile of land, estimated for 2012) according to this reclassification table file.
- View in a new window Be sure to let the user call the new field whatever s/he pleases.
- Let block groups whose densities do not fall within the ranges specified by the reclassification table be classified 9999.

If you were to then take the shapefile your code/tool/service outputted and manually map the reclassified field, and if you were to choose 'unique values' symbology, and if you were to zoom in on Seattle a little, you should find that your results look like this.Preview the documentView in a new window Look carefully--subtle differences from your outputs may indicate that you've failed to handle some cases properly!

In [3]:
import sys
sys.path.append(r'C:\Program Files (x86)\ArcGIS\Desktop10.3\bin')
sys.path.append(r'C:\Program Files (x86)\ArcGIS\Desktop10.3\arcpy')
sys.path.append(r'C:\Program Files (x86)\ArcGIS\Desktop10.3\ArcToolbox\Scripts')

# Because Workspace
import arcpy     
import os
arcpy.env.workspace="F:\\UW\\GEOG458 Python\\Lab 2_ToolsService\\workspace\\"  

### Flexable code that allows for reclassification, according to flex critearia (names)

In [4]:
# Nested Cursor Paramaters
arcpy.env.workspace = "F:\\UW\\GEOG458 Python\\Lab 2_ToolsService\\workspace"
outputfolder = "F:\\UW\\GEOG458 Python\\Lab 2_ToolsService\\workspace\\output\\"
infile = "F:\\UW\\GEOG458 Python\\Lab 2_ToolsService\\workspace\\King.shp"
fieldy = "PopDens12"
mytable = "F:\\UW\\GEOG458 Python\\Lab 2_ToolsService\\workspace\\ReclassTableExample.dbf"
lower = "lowerbound"
upper = "upperbound"
value = "value"
output = "F:\\UW\\GEOG458 Python\\Lab 2_ToolsService\\workspace\\output\\hello.shp"
out_name = "ranking"
novalue = "whereami"

In [24]:
# Checking fieldnames in king shapefile. 
fieldlist = arcpy.ListFields(r"F:\UW\GEOG458 Python\Lab 2_ToolsService\workspace\ReclassTableExample.dbf", "")
for field in fieldlist:
    print field.name 
print arcpy.GetMessages() 

OID
lowerbound
upperbound
value



In [ ]:
# Copy shapefiles 
arcpy.CopyFeatures_management (infile, output)

In [7]:
# Add a field
arcpy.AddField_management (output, out_name, "FLOAT")

<Result 'F:\\UW\\GEOG458 Python\\Lab 2_ToolsService\\workspace\\output\\hello.shp'>

In [6]:
# Working with nested cursors
with arcpy.da.SearchCursor(output, [fieldy, out_name]) as Cursource:
    for srow in Cursource:
        with arcpy.da.SearchCursor(mytable, [upper, lower, value]) as Curtable:
            for trow in Curtable:
                if arcpy.sa.GreaterThanEqual(trow[0],srow[0]) and arcpy.sa.LessThanEqual(trow[1],srow[0]):
                        srow[1] = trow[2]
                        Cursource.updateRow(srow)
                else:
                    srow[1] = notfoundvalue
                    Cursource.updateRow(srow)
print arcpy.GetMessages() 

RuntimeError: ERROR 000824: The tool is not licensed.

In [12]:
# Working with nested cursors
with arcpy.da.UpdateCursor(output, [fieldy, out_name]) as Cursor:
    for row in Cursor:
        with arcpy.da.SearchCursor(mytable, [upper, lower, value]) as Cursor2:
            for row2 in Cursor2:
                if row[0] <= row2[0] and row[0] >= row2[1]:
                    row[1] = row2[2]
                else:
                    out_name = novalue
                    # I tried using update row but it said that da.SearchCursor had no attribute 'updateRow'
print arcpy.GetMessages() 

RuntimeError: A column was specified that does not exist.